### Install sagemaker-experiments

In [1]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

In [2]:
import sagemaker
import boto3

session = sagemaker.Session()
sm = boto3.Session().client('sagemaker')
role = sagemaker.get_execution_role()

### Split Data

In [14]:
import os

os.chdir('/home/sagemaker-user/aws_sagemaker/scikit-learn/classification/iris_parameterized')

FileNotFoundError: [Errno 2] No such file or directory: '/home/sagemaker-user/aws_sagemaker/scikit-learn/classification/iris_parameterized'

In [6]:
import pandas as pd

RANDOM_STATE = 99

DATA_FILE = './data/iris.csv'

# load csv in memory
data = pd.read_csv(DATA_FILE)
data.head()

FileNotFoundError: [Errno 2] File ./data/iris.csv does not exist: './data/iris.csv'

In [4]:
# split data into test and training
train_data = data.sample(frac=0.8, random_state=RANDOM_STATE)
test_data = data.drop(train_data.index)

test_data = test_data.drop(['class'],axis=1)
test_data.head()

NameError: name 'data' is not defined

In [5]:
# put train data into csv
TRAIN_DATA_FILE = './data/train_data.csv'
train_data.to_csv(TRAIN_DATA_FILE, index=False, header=True)

# put test data into csv
TEST_DATA_FILE = './data/test_data.csv'
test_data.to_csv(TEST_DATA_FILE, index=False, header=True)

### Push data to S3 bucket

#### Define S3 Paths

In [6]:
BUCKET_NAME = 'snowflake-getting-started'
BASE_PREFIX = 'iris'

INPUT_DATA_PREFIX = BASE_PREFIX+'/data/input'
TRAIN_DATA_PREFIX = BASE_PREFIX+'/data/input/train'
TEST_DATA_PREFIX = BASE_PREFIX+'/data/input/test'

EXPERIMENTS_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/experiments'
#DEBUGGING_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/output/debugging'
#MODELS_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/output/models'

print ('Experiment metadata would be published at -',EXPERIMENTS_OUTPUT_LOC)

Experiment metadata would be published at - s3://snowflake-getting-started/iris/experiments


#### Upload data to S3

In [7]:
print ('Uploading train data to s3')

s3_input_data_path = session.upload_data(path=DATA_FILE, 
                           bucket=BUCKET_NAME, 
                           key_prefix=INPUT_DATA_PREFIX)

print ('Input data uploaded to -', s3_input_data_path)

Uploading train data to s3
Input data uploaded to - s3://snowflake-getting-started/iris/data/input/iris.csv


### Data Pre-Processing

#### Pre-processing tasks

    This is where we would do data pre-processing and convert intput data into some features. These features would then be uploaded to S3 and would be
    used for model training.

#### Create Tracker for Pre-Processing

    This is where we create a tracker for pre-processing job and add to it the following:
    1 - Input data location (S3)
    2 - Output data / Features Location (S3)
    3 - Significant parameters from pre-processing.
        

In [8]:
from smexperiments.tracker import Tracker

with Tracker.create(display_name="Preprocessing", sagemaker_boto_client=sm) as tracker:
    tracker.log_parameters({
        "pre_processing_param": 'pre_processing_param_value'
    })
    # we can log the s3 uri to the dataset we just uploaded
    tracker.log_input(name="iris-dataset", media_type="s3/uri", value=s3_input_data_path)
    tracker.log_output(name="iris-dataset", media_type="s3/uri", value=s3_input_data_path)

### Step 1 - Setup an Experiment

### Create an Experiment

In [18]:
from smexperiments.experiment import Experiment
import time

EXPERIMENT_NAME=f"experiment-iris-classification-model-{int(time.time())}"

iris_experiment = Experiment.create(
    experiment_name=EXPERIMENT_NAME,
    description="Classification of iris flowers", 
    sagemaker_boto_client=sm)

EXP_DEBUGGING_OUTPUTS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/debugging'
EXP_TRAINED_MODELS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/trained_models'
BASE_JOB_NAME=BASE_PREFIX+'/experiments'+EXPERIMENT_NAME+'/code'

print ('Experiment debugging data available at -',EXP_DEBUGGING_OUTPUTS)
print ('Experiment trained moddels available at -',EXP_TRAINED_MODELS)

Experiment debugging data available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595794471/debugging
Experiment trained moddels available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595794471/trained_models


### Step 2 - Track Experiment

### Now create a Trial for each training run to track the it's inputs, parameters, and metrics.

In [19]:
from sagemaker.sklearn import SKLearn
from smexperiments.trial import Trial

from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig, CollectionConfig

# we are taking the trial component which was created earlir for preprocessing. since for different training trails
# our pre-processing trial is the same hence we just attach that preprocessing component to this trial.
preprocessing_trial_component = tracker.trial_component

save_interval ='1'

for i, num_max_iter in enumerate([5,10]):
    
    # create trial
    trial_name = f"trial-iris-classification-model-{num_max_iter}-max-iter-{int(time.time())}"
    
    iris_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=iris_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    
    # associate the proprocessing trial component with the current trial
    iris_trial.add_trial_component(preprocessing_trial_component)
    
    executor = SKLearn(entry_point='train.py',
                      train_instance_type='ml.c4.xlarge',
                      sagemaker_session = session,
                      role = role,
                      hyperparameters = {
                          'max_iter':num_max_iter,
                          'class_weight':'balanced'
                      },
                      input_mode='File',
                      base_job_name=BASE_JOB_NAME,
                      metric_definitions=[
                        {'Name':'test:f1-score', 'Regex':'Test F1-Score: (.*)'},
                        {'Name':'test:accuracy', 'Regex':'Test Accuracy: (.*)'}
                      ],
                      enable_sagemaker_metrics=True,
                      debugger_hook_config=DebuggerHookConfig(
                                  s3_output_path=EXP_DEBUGGING_OUTPUTS, 
                                  # Required - See https://github.com/awslabs/sagemaker-debugger/blob/master/docs/api.md#built-in-collections for supported collections
                                  collection_configs=[ 
                                      CollectionConfig( name="test-f1-score-collection", 
                                                        parameters={
                                                            'include_regex': 'Test F1-Score: (.*)',
                                                            "save_interval": save_interval 
                                                        } 
                                                      ), 
                                      CollectionConfig( name="test-accuracy-collection", 
                                                        parameters={
                                                            'include_regex': 'Test Accuracy: (.*)',
                                                            "save_interval": save_interval 
                                                        } 
                                                      ), 
                                      #CollectionConfig( name="feature_importance", parameters={ "save_interval": save_interval } ), 
                                  ], 
                                ),
                       # Configure debugger rule
                       rules=[
                            Rule.sagemaker(                                      
                                rule_configs.loss_not_decreasing(),                  
                                rule_parameters={
                                    "collection_names": "metrics"
                                },
                            ),
                        ],                     
                      output_path = EXP_TRAINED_MODELS
                )
    
    iris_training_job_name = "training-job-iris-classification-model-{}".format(int(time.time()))
    executor.fit(
        inputs={'training': s3_input_data_path},
        job_name=iris_training_job_name,
        experiment_config={
            "TrialName": iris_trial.trial_name,
            "TrialComponentDisplayName": "Training",
        },
        logs=True,
        wait=True
    )
    
    # give it a while before dispatching the next training job
    time.sleep(2)

### Compare the model training runs for an experiment

Now we will use the analytics capabilities of Python SDK to query and compare the training runs for identifying the best model produced by our experiment. You can retrieve trial components by using a search expression.

### Some Simple Analyses

In [15]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

In [16]:
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.session import Session

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(boto3.Session(), sm), 
    experiment_name=iris_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.test:f1-score.max",
    metric_names=['test:f1-score'],
    sort_order="Descending",    
    #metric_names=['test:f1-score'],
    parameter_names=['max_iter', 'class_weight']
)
trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,class_weight,max_iter,test:f1-score - Min,test:f1-score - Max,test:f1-score - Avg,test:f1-score - StdDev,test:f1-score - Last,test:f1-score - Count
0,training-job-iris-classification-model-1595793...,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,"""balanced""",5.0,1.000000,1.000000,1.000000,0.0,1.000000,1
1,training-job-iris-classification-model-1595794...,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,"""balanced""",10.0,0.947368,0.947368,0.947368,0.0,0.947368,1


In [17]:
lineage_table = ExperimentAnalytics(
    sagemaker_session=Session(boto3.Session(), sm), 
    search_expression={
        "Filters":[{
            "Name": "Parents.TrialName",
            "Operator": "Equals",
            "Value": 'iris-training-job-20-max-iter-1595707533'
        },
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }]
    },
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_table.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,class_weight,max_iter,sagemaker_container_log_level,...,test:f1-score - Avg,test:f1-score - StdDev,test:f1-score - Last,test:f1-score - Count,test:accuracy - Min,test:accuracy - Max,test:accuracy - Avg,test:accuracy - StdDev,test:accuracy - Last,test:accuracy - Count
0,iris-training-job-1595707533-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",20.0,20.0,...,0.947368,0.0,0.947368,1,0.947368,0.947368,0.947368,0.0,0.947368,1


### Evaluating Debugging Outputs

In [ ]:
!pip install smdebug

In [ ]:
import smdebug
from smdebug.trials import create_trial

trial = create_trial(EXP_DEBUGGING_OUTPUTS)

##### Plot Metrics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re

def get_data(trial, tname):
    """
    For the given tensor name, walks though all the iterations
    for which you have data and fetches the values.
    Returns the set of steps and the values.
    """
    tensor = trial.tensor(tname)
    steps = tensor.steps()
    vals = [tensor.value(s) for s in steps]
    return steps, vals

def plot_collection(trial, collection_name, regex='.*', figsize=(8, 6)):
    """
    Takes a `trial` and a collection name, and 
    plots all tensors that match the given regex.
    """
    fig, ax = plt.subplots(figsize=figsize)
    sns.despine()

    tensors = trial.collection(collection_name).tensor_names

    for tensor_name in sorted(tensors):
        if re.match(regex, tensor_name):
            steps, data = get_data(trial, tensor_name)
            ax.plot(steps, data, label=tensor_name)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Iteration')

In [ ]:
plot_collection(trial, "metrics")

##### Plot Feature Importance

In [ ]:
def plot_feature_importance(trial, importance_type="weight"):
    SUPPORTED_IMPORTANCE_TYPES = ["weight", "gain", "cover", "total_gain", "total_cover"]
    if importance_type not in SUPPORTED_IMPORTANCE_TYPES:
        raise ValueError(f"{importance_type} is not one of the supported importance types.")
    plot_collection(
        trial,
        "feature_importance",
        regex=f"feature_importance/{importance_type}/.*")
    
 plot_feature_importance(trial)    

### Step - Real Time Predictions

#### Deploy Model to an Endpoint

In [ ]:
predictor = executor.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
print('\nModel Deployed!')

print (predictor.endpoint)

#### Realtime Inference

    Lookup the predictor via the endpoint name & call predict on it

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(DATA_FILE,engine='python')
X = data.iloc[:,1:5]
y = data.iloc[:,5]

train_x, test_x, train_y, test_y = train_test_split(X,y)
pred_y = predictor.predict(test_x)
print (pred_y)
print (test_y)